In [1]:
import pandas as pd
hosp = '/Users/mariama/Documents/GitHub/small-things/urgent_care_project/hosp.csv'
hospital = pd.read_csv(hosp, encoding = "ISO-8859-1")
del(hospital['Unnamed: 0']) #delete the numbered rows that R added in. 

pop_means = 'pop_means_tract.csv'
center = pd.read_csv(pop_means, encoding = "ISO-8859-1")

In [2]:
er = hospital[hospital.TYPE.isin(['GENERAL ACUTE CARE','CRITICAL ACCESS'])]

In [3]:
# Taken from: https://gist.github.com/rochacbruno/2883505 (Thanks!!)

# Haversine formula example in Python
# Author: Wayne Dyck

import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [4]:
hosp_latlon = pd.DataFrame(columns = ['latlon'])
hll = er[['LAT','LON']].values.tolist()
hosp_latlon['latlon'] = hll

ct_latlon = pd.DataFrame(columns = ['latlon'])
ctll = center[['LATITUDE','LONGITUDE']].values.tolist()
ct_latlon['latlon'] = ctll

In [5]:
#hosp_list = []
def get_nearest_hosp(ct_locs):
    hosp_locs = hosp_latlon["latlon"]
    dist_list = []
    for item in hosp_locs: 
        dist_list.append(distance(item, ct_locs))
    temp = pd.DataFrame(dist_list)
    #closest_hospital = er.iloc[temp.idxmin()]
    #closest_hospital = temp.min()
    closest_hospital_index = temp.idxmin()
    return closest_hospital_index


def get_nearest_hosp_dist(ct_locs):
    hosp_locs = hosp_latlon["latlon"]
    dist_list = []
    for item in hosp_locs: 
        dist_list.append(distance(item, ct_locs))
    temp = pd.DataFrame(dist_list)
    closest_hospital_dist = temp.min()
    return closest_hospital_dist

In [6]:
import time
start_time = time.time()

ct_ll  = ct_latlon.head()
ct_ll['latlon'].apply(get_nearest_hosp)
ct_ll['latlon'].apply(get_nearest_hosp_dist)

print("My program took", time.time() - start_time, "to run")

My program took 0.12011599540710449 to run


In [ ]:
center['hosp_index'] = ct_latlon['latlon'].apply(get_nearest_hosp)
center['dist'] = ct_latlon['latlon'].apply(get_nearest_hosp_dist)

In [ ]:
er = er.reset_index(drop=True)
fullset = pd.merge(center, er, left_on = "hosp_index", right_index=True, how = 'left')